In [1]:
# Main
import numpy as np
import pandas as pd
import datetime as dt
from statsmodels.tsa.tsatools import lagmat

# Modeling
import statsmodels.api as sm
from statsmodels.tsa.tsatools import add_trend
from statsmodels.regression.linear_model import OLS
from statsmodels.tsa.ar_model import ar_select_order
from statsmodels.tsa.stattools import adfuller

# Reporting
from statsmodels.iolib.summary2 import summary_col
from IPython.display import display, HTML

# Plotting
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

In [2]:
def saveToExcel(dataframe, path, def_properties={}):
    # Setup
    writer = pd.ExcelWriter(path)
    dataframe.style.set_properties(**def_properties).to_excel(writer)

    def dummyCriteria(f): return {'type': 'cell',
                                  'criteria': '!=', 'value': 0, 'format': f}

    def excel_colnum_string(n):
        string = ""
        while n > 0:
            n, remainder = divmod(n - 1, 26)
            string = chr(65 + remainder) + string
        return string

    # Accessing
    workbook = writer.book
    worksheet = writer.sheets['Sheet1']
    colLetter = excel_colnum_string(dataframe.shape[1]+1)
    tableRange = 'A1:'+colLetter+str(dataframe.shape[0]+1)
    header = 'A1:'+colLetter+'1'
    footer = 'A'+str(dataframe.shape[0]+1) + \
        ':'+colLetter+str(dataframe.shape[0]+1)
    indexCol = 'A1:'+'A'+str(dataframe.shape[0]+1)

    # Styles
    defaultFormat = {'font_name': 'Calibri', 'font_size': 8,
                     'bg_color': '#FFFFFF',
                     'align': 'center',
                     'valign': 'vcenter',
                     'border': 0,
                     'shrink': True,
                     'text_wrap': True}
    columnFormat = workbook.add_format(defaultFormat)
    columnFormat.set_font_size(8)
    headerFormat = workbook.add_format(
        {**defaultFormat, **{'top': 5, 'bottom': 2, 'bold': True}})
    footerFormat = workbook.add_format({**defaultFormat, **{'bottom': 5}})
    indexFormat = workbook.add_format({**defaultFormat, **{'align': 'left'}})

    # Applying Styles
    worksheet.conditional_format(tableRange, dummyCriteria(columnFormat))
    worksheet.conditional_format(header, dummyCriteria(headerFormat))
    worksheet.conditional_format(footer, dummyCriteria(footerFormat))
    worksheet.conditional_format(indexCol, dummyCriteria(indexFormat))
    writer.save()
    print('Table saved to:', path)

In [3]:
data = pd.read_pickle('data/dataset2000.pkl')

# Changing data
dataLogs = data.apply(np.log, axis=1)
dataPtC = dataLogs.diff()[1:]

In [4]:
# This replicates the results above with the ADF class
lags = 1 # dynamically scalable
diff = dataLogs['Armenia'].diff(1)
d_y = diff[1+lags:]
y_1 = dataLogs['Armenia'].shift(1).rename('Level.L1')
d_y_lags = pd.DataFrame(lagmat(diff, maxlag=lags), index=diff.index,
                        columns=['Diff.L'+str(i) for i in range(1, lags+1)])
X = sm.add_constant(pd.concat([y_1, d_y_lags], axis=1))[1+lags:]
res = OLS(d_y, exog=X).fit()
res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Armenia   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.034
Method:                 Least Squares   F-statistic:                     5.441
Date:                Tue, 03 Aug 2021   Prob (F-statistic):            0.00487
Time:                        02:43:54   Log-Likelihood:                 598.30
No. Observations:                 250   AIC:                            -1191.
Df Residuals:                     247   BIC:                            -1180.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0736      0.043      1.715      0.088      -0.011       0.158
Level.L1      -0.0151      0.009     -1.699      0.090      -0.033       0.002
Diff.L1        0.1820      0.063      2.895      0.004       0.058       0.306
==============================================================================
Omnibus:                      126.986   Durbin-Watson:                   1.973
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2159.310
Skew:                          -1.563   Prob(JB):                         0.00
Kurtosis:                      17.054   Cond. No.                         221.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Model overview:

$\Delta y_t= \alpha_0 + \sum_{j=1}^{q} \alpha_j \Delta y_{t-j} + \beta_1 y_{t-1} + (\beta_2-\beta_1)[t\geq t^*] y_{t-1} + \epsilon_t$

When: lags = 1; EAEU = 1*$[t\geq t^*]$; then:

$\Delta y_t= \alpha_0 + \alpha_1 \Delta y_{t-1} + \beta_1 y_{t-1} + (\beta_2-\beta_1)\small{\textrm{EAEU}}y_{t-1} + \epsilon_t$

In [9]:
def mainModel(endog, t_star, maxlags=1, trend=False):
    lags = len(ar_select_order(endog, maxlags, old_names=False).ar_lags)
    diff = endog.diff(1)
    d_y = diff[1+lags:]
    EAEU = (endog.index > t_star).astype(int)
    y_1 = endog.shift(1).rename('ρ₁')
    y_1_post = (y_1*EAEU).rename('α₁')
    d_y_lags = pd.DataFrame(lagmat(diff, maxlag=lags), index=diff.index,
                            columns=[f'𝛾{str(chr(8320+i))}' for i in range(1, lags+1)])
    X = sm.add_constant(pd.concat([y_1, y_1_post, d_y_lags], axis=1))[1+lags:]
    if trend:
        X = add_trend(X, 'ct')
    res = OLS(d_y, exog=X).fit()
    return res

In [10]:
# ADF test with post-EAEU dummy
log_y = dataLogs['Kazakhstan']
t_star = '2015-01-01'
lags = 12

results = mainModel(log_y, t_star, lags)
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Kazakhstan   R-squared:                       0.167
Model:                            OLS   Adj. R-squared:                  0.154
Method:                 Least Squares   F-statistic:                     12.24
Date:                Tue, 03 Aug 2021   Prob (F-statistic):           4.35e-09
Time:                        02:50:59   Log-Likelihood:                 584.44
No. Observations:                 249   AIC:                            -1159.
Df Residuals:                     244   BIC:                            -1141.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.2255      0.064      3.528      0.000       0.100       0.351
ρ₁            -0.0470      0.013     -3.503      0.001      -0.073      -0.021
α₁            -0.0030      0.001     -3.606      0.000      -0.005      -0.001
𝛾₁             0.3289      0.062      5.285      0.000       0.206       0.452
𝛾₂            -0.0159      0.063     -0.255      0.799      -0.139       0.107
==============================================================================
Omnibus:                      196.616   Durbin-Watson:                   2.001
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4537.012
Skew:                          -2.850   Prob(JB):                         0.00
Kurtosis:                      23.120   Cond. No.                         246.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
# Running main model for all countries
t_star = '2015-01-01'
lags = 12
modelResults = []

for country in dataLogs:
    modelResults.append(mainModel(dataLogs[country], t_star, lags))

In [12]:
models = modelResults
modelNames = list(dataLogs.columns)

view = summary_col(models, model_names=modelNames, 
                   stars=True, float_format='%.3f')
view.add_title('Main Model Results')
view.extra_txt = ['* p<.1, ** p<.05, ***p<.01']
summarayTable = view.tables[0]
summarayTable[:8]

,Armenia,Belarus,Kazakhstan,Kyrgyz Republic,Russian Federation
R-squared,0.043,0.129,0.167,0.037,0.295
R-squared Adj.,0.028,0.111,0.154,0.026,0.278
const,0.066,0.161**,0.225***,0.093,0.140***
,(0.048),(0.077),(0.064),(0.059),(0.049)
α₁,-0.000,-0.003**,-0.003***,-0.001,-0.001
,(0.001),(0.002),(0.001),(0.001),(0.001)
ρ₁,-0.013,-0.037**,-0.047***,-0.019,-0.026***
,(0.010),(0.018),(0.013),(0.012),(0.009)


In [14]:
summarayTable[:8].to_excel('Outputs/TAR_logReport(C).xlsx')

In [15]:
# Running main model for all countries
t_star = '2015-01-01'
lags = 12
modelResults = []

for country in dataLogs:
    modelResults.append(mainModel(dataLogs[country], t_star, lags, trend=True))

In [19]:
models = modelResults
modelNames = list(dataLogs.columns)

view = summary_col(models, model_names=modelNames, 
                   stars=True, float_format='%.3f')
view.add_title('Main Model Results')
view.extra_txt = ['* p<.1, ** p<.05, ***p<.01']
summarayTable = view.tables[0]
summarayTable[:10]

,Armenia,Belarus,Kazakhstan,Kyrgyz Republic,Russian Federation
R-squared,0.051,0.147,0.198,0.058,0.302
R-squared Adj.,0.032,0.126,0.181,0.042,0.282
const,0.150*,0.427***,0.328***,0.267***,0.005
,(0.076),(0.141),(0.071),(0.096),(0.102)
trend,0.000,-0.000**,0.000***,0.000**,-0.000
,(0.000),(0.000),(0.000),(0.000),(0.000)
α₁,-0.001,-0.002,-0.007***,-0.002**,0.003
,(0.001),(0.002),(0.002),(0.001),(0.002)
ρ₁,-0.032*,-0.094***,-0.071***,-0.057***,0.002
,(0.017),(0.031),(0.015),(0.021),(0.021)


In [20]:
summarayTable[:10].to_excel('Outputs/TAR_logReport(C&T).xlsx')